In [ ]:
import numpy as np  
import pandas as pd  

import matplotlib.pyplot as plt # For plots
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import datetime
import re
pd.set_option('display.max_columns', None)

In [ ]:
df_appr_full = pd.read_pickle("./IN_dfs/df_appr_full_raw.pkl")
df_comp_full = pd.read_pickle("./IN_dfs/df_comp_full_raw.pkl")
print("Appriasal Shape", df_appr_full.shape)
print("Comp Shape", df_comp_full.shape)

In [ ]:
# After changing which df to process Run all cells below this one.

In [ ]:
#Select which dataframe to process. Comment out the df you don't need. Run the entire notebook twice to save both dataframes.

#which_df = "APPRAISAL"
which_df = "COMPARABLES"

show_analyse_column_output = False # False surpresses output and speeds out running of the notebook

if which_df == "APPRAISAL":
    df_used = df_appr_full.copy()
elif which_df == "COMPARABLES":
    df_used = df_comp_full.copy()
    cols = df_appr_full.columns
    df_used.columns = cols
else:
    raise ValueError("############Incorrect dataframe selected#############")
    
df_new_used = pd.DataFrame() # new dataframe post data cleaning
    

In [ ]:
%%time
for c, a in zip(df_comp_full.columns, df_appr_full.columns):
    print(c,"///",a)

In [ ]:
# Create some smaller dataframes
# using the train test split function
#Y_90pc, Y_10pc = train_test_split(df_appr_full,
#                                   random_state=104, 
#                                   test_size=0.1, 
#                                   shuffle=True)
#Y_6m_10k, Y_10k = train_test_split(df_appr_full,
#                                   random_state=104, 
#                                   test_size=10000, 
#                                   shuffle=True)

#Y_6m_100k, Y_100k = train_test_split(df_appr_full,
#                                   random_state=104, 
#                                   test_size=100000, 
#                                   shuffle=True)

In [ ]:
df_used.columns

## Categorize each column for batch processing and wite helper funcitons

In [ ]:
columns_binary = ['LOCRTGNEUTRAL_SUBJ', 'LOCRTGBENEFICIAL_SUBJ',
       'LOCRTGADVERSE_SUBJ', 'LOCRESIDENTIAL_SUBJ', 'LOCINDUSTRIAL_SUBJ',
       'LOCCOMMERCIAL_SUBJ', 'LOCBUSYROAD_SUBJ', 'LOCWATERFRONT_SUBJ',
       'LOCGOLFCOURSE_SUBJ', 'LOCADJPARK_SUBJ', 'LOCADJPOWERLINES_SUBJ',
       'LOCLANDFILL_SUBJ', 'LOCPUBLICTRANS_SUBJ','VIEWRTGNEUTRAL_SUBJ', 'VIEWRTGBENEFICIAL_SUBJ', 'VIEWRTGADVERSE_SUBJ',
       'VIEWTYPEWATER_SUBJ', 'VIEWTYPEPASTORAL_SUBJ', 'VIEWTYPEWOODS_SUBJ',
       'VIEWTYPEPARK_SUBJ', 'VIEWTYPEGOLFCOURSE_SUBJ',
       'VIEWTYPECITYSKYLINE_SUBJ', 'VIEWTYPEMOUNTAIN_SUBJ',
       'VIEWTYPERESIDENTIAL_SUBJ', 'VIEWTYPECITYSTREET_SUBJ',
       'VIEWTYPEINDUSTRIAL_SUBJ', 'VIEWTYPEPOWERLINES_SUBJ',
       'VIEWTYPELIMITED_SUBJ' ] # convert to 0/1 columns
columns_quality = ['QUALITYOFCONSTQ1_SUBJ','QUALITYOFCONSTQ2_SUBJ', 'QUALITYOFCONSTQ3_SUBJ','QUALITYOFCONSTQ4_SUBJ', 
                   'QUALITYOFCONSTQ5_SUBJ', 'QUALITYOFCONSTQ6_SUBJ'] # convert to 1 continuous column?
columns_condition = ['CONDITIONC1_SUBJ','CONDITIONC2_SUBJ', 'CONDITIONC3_SUBJ', 'CONDITIONC4_SUBJ',
                       'CONDITIONC5_SUBJ', 'CONDITIONC6_SUBJ'] # convert to 1 continuous column?
columns_numeric = ['TOTALRMS_SUBJ', 'BDRMS_SUBJ',
        'BLGRDTOTALSQFT_SUBJ', 'BLGRDFINISHSQFT_SUBJ',
       'BLGRDRECRMS_SUBJ', 'BLGRDBEDRMS_SUBJ', 
       'BLGRDOTHERRMS_SUBJ', 'GROSSLIVINGAREA_SUBJ'] 
columns_age = ['ACTUALAGE_SUBJ']
columns_bathrooms = ['BATHS_SUBJ','BLGRDBATHRMS_SUBJ'] #split into two

### Columns below need most attention ###
date_columns = [  'SALEDATE_SUBJ', 'COMPSALEDATE'] # almost doen - need year
site_columns = ['SITE_SUBJ']

special_columns = [ 'DESIGNSTYLE_SUBJ', 'HEATINGCOOLING_SUBJ', 'ENERGYEFF_SUBJ',
       'GARAGECARPORT_SUBJ', 'PORCHPATIODECK_SUBJ' ]

lat_long_columns = ['APPRLATITUDE_SUBJ', 'APPRLONGITUDE_SUBJ',]

identifier_columns = ['SUBJ_APPR_ID','COMPNUM', 'ADDRESS1_SUBJ', 'CITY_SUBJ', 'STATE_SUBJ',
       'ZIPCODE_SUBJ', 'COUNTY_SUBJ']

ignored_columns = ['LOCATIONDESC_SUBJ', 'VIEWDESC_SUBJ','BASEMENT_SUBJ','FINISHEDRMSBLWGRD_SUBJ']
# locationdesc, site and viewdesc already encoded into binary columns in data, can ignore them
# 'BASEMENT_SUBJ','FINISHEDRMSBLWGRD_SUBJ' also already ensoded into columns




In [ ]:
# Check if the individual lists above have all the columns from main dataframe
columns_being_analysed = [ *identifier_columns, *columns_binary,*columns_quality, *columns_condition, *columns_numeric, *site_columns,
                                *columns_bathrooms, *special_columns, *date_columns, *ignored_columns,
                                *lat_long_columns, *columns_age]

missing_cols = [item for item in list(df_appr_full.columns) 
                if item not in columns_being_analysed ]

if len(missing_cols)== 0:
    print("No missing columns")
else:
     print("#### THERE ARE MISSING COLUMNS ####\n",missing_cols)   

duplicates = [i for i in columns_being_analysed if columns_being_analysed.count(i) > 1]
unique_duplicates = list(set(duplicates))
print("\nDuplicate columns:\n\n", unique_duplicates)
 

## Helper Functions

In [ ]:
def column_name_adjuster(columns_list, source_df):
    # takes in a list of columns as strings and source df and figures out if the columns are in it.
    # returns column list with names either with or without _SUBJ. Agnostic of whether using appr or comp frames
    if all(item in source_df.columns for item in columns_list):
        return (columns_list)
    else:
        if "_SUBJ" in columns_list[0]:
            columns_list1 = [elem[:-5] for elem in columns_list]
            return (columns_list1)
        else:
            columns_list2 = [elem + "_SUBJ" for elem in columns_list]
            return (columns_list2)

In [ ]:
def column_summary(col,dataframe):
    # column summary - pass col name as string + dataframe. Receive sorted occurence count and %
    # TODO nan handling - atm not showing
    pd.set_option('display.max_rows', None)

    counts = dataframe[col].value_counts()
    percs = dataframe[col].value_counts(normalize=True).mul(100).round(3).astype(str) + '%'
    return pd.concat([counts,percs], axis=1, keys=['count', 'percentage'])


def analyse_column_list(col_lst, df,show=True):
    if show == False:
        return None
    else:
        col_lst1 = column_name_adjuster(col_lst, df)
        for column in col_lst1:
            print("\n*********",'\033[1m' + column + '\033[0m')
            display(column_summary(column,df))

### Check and transform identifier columns

In [ ]:
temp = pd.DataFrame()
temp[identifier_columns] = df_used[identifier_columns].replace({'[NULL]': np.nan})
#print(identifier_columns)
temp['SUBJ_APPR_ID'] = temp['SUBJ_APPR_ID'].astype("Int64")
temp['COMPNUM'] = temp['COMPNUM'].astype("Int64")
analyse_column_list(identifier_columns,temp.head(1000),show=False)

df_new_used[identifier_columns] = temp[identifier_columns]

### Check and transform binary columns 

In [ ]:
# check all binary columns 
analyse_column_list(columns_binary, df_used,show=False)

In [ ]:
# check all quality columns # might want to convert these to continuous #TODO?
analyse_column_list(columns_quality, df_used,show=False)
analyse_column_list(columns_condition, df_used,show=False)

In [ ]:
# encode all binary columns

df_new_used[columns_binary] = df_used[columns_binary].replace({'Y': 1,'[NULL]': 0})
df_new_used[columns_quality] = df_used[columns_quality].replace({'Y': 1,'[NULL]': 0})
df_new_used[columns_condition] = df_used[columns_condition].replace({'Y': 1,'[NULL]': 0})

analyse_column_list([*columns_binary,*columns_quality,*columns_condition],df_new_used,show=False)


### Check and transform latitude and longitude

In [ ]:
df_temp4 = pd.DataFrame()
for j in lat_long_columns:
    df_temp4[j]=pd.to_numeric(df_used[j],errors="ignore",downcast="float")
    df_temp4[j] = df_temp4[j].replace([0.,"[NULL]"], np.nan)
    
df_new_used[lat_long_columns] = df_temp4[lat_long_columns]    

### Check and transform numeric columns 

In [ ]:
# check all numeric columns PRoposing to replace all NULLs with zero TODO
analyse_column_list(columns_numeric,df_used,show=False)

In [ ]:
# Clean numeric cols
for num_col in columns_numeric:
    df_new_used[num_col] = df_used[num_col].str.replace(r'[^\d.]+', '', regex=True)
    df_new_used[num_col] = df_new_used[num_col].str.replace(r'[^\d.]+', '', regex=True) #remove character if not decimal digit or .
    df_new_used[num_col] = df_new_used[num_col].str.replace(r'^(0*)', '',regex=True) #remove leading zero
    #df_new_used[df_new_used["TOTALRMS_SUBJ"]==""] = np.NaN
    df_new_used[num_col]=pd.to_numeric(df_new_used[num_col],errors="ignore",downcast="integer")
    #TODO fill NA
    #df_new_used['TOTALRMS_SUBJ'] = df_new_used['TOTALRMS_SUBJ'].astype(int)

analyse_column_list(columns_numeric,df_new_used,show=False)

### Check and transform date columns

In [ ]:
%%time
# Check date columns
df_temp=pd.DataFrame()
df_temp[date_columns] = df_used[date_columns]

#clean up the extracted_number field
for c in date_columns:
    if c == "SALEDATE_SUBJ":
        lst = df_temp[c].unique()
        print(c, "length", len(lst))
        lst_num = []
        for no in lst:
            try:
                #print(no)
                test = datetime.datetime.strptime(no, '%m/%d/%Y')
                lst_num.append(test)
                #print(test)
            except Exception as e: 
                if no=="[NULL]":
                    lst_num.append(np.nan)
                else:
                    print("error",no)
        df_new_used[c] = df_temp[c]
    if c == "COMPSALEDATE":
        lst = df_temp[c].unique()
        print(c, "length", len(lst))
        lst_num = []
        for no in lst:
            try:
                #print(no)
                test = datetime.datetime.strptime(no, '%m/%y')
                lst_num.append(test)
                #print(test)
            except Exception as e: 
                try:
                    #print(no)
                    test = datetime.datetime.strptime(no, '%m/%d/%Y')
                    lst_num.append(test)
                    #print(test)
                except Exception as e:
                    if no=="[NULL]":
                        lst_num.append(np.nan)
                    else:
                        lst_num.append(np.nan)
                        print("replaced with NaN:",no)
    date_dict = dict(zip(lst, lst_num))
   # df_new_used[c] = df_used[c].replace(date_dict)
    df_new_used[c] = df_used[c].map(date_dict.get)
    
df_new_used["SALEDATE_SUBJ"] = pd.to_datetime(df_new_used["SALEDATE_SUBJ"], errors = 'coerce')
df_new_used["COMPSALEDATE"] = pd.to_datetime(df_new_used["COMPSALEDATE"], errors = 'coerce')

print("######Date columns transformed")

In [ ]:
df_new_used.head()

### Check and transform age column

In [ ]:
# clean up age
analyse_column_list(columns_age,df_used,show=False)

In [ ]:
# Clean up ACTUALAGE_SUBJ #TODO if age is > 1900, deduct year of appraisal to get age. Clarify age>>100

lst = df_used['ACTUALAGE_SUBJ'].unique()

import re
lst_num = []
for no in lst:
        temp_list = re.findall(r'\d+', no)
        
        if len(temp_list)==0:
            x =(np.NaN)
        elif len(temp_list)==1:
            x =(temp_list[0])
        else:
            x = (temp_list[-1])
        
        if pd.isnull(x):
            lst_num.append(x)
        
        elif len(x) >1:
            x = x.lstrip('0')
            if x =="":
                x='0'
            lst_num.append(int(x))
        else:
            lst_num.append(int(x))
age_dict = dict(zip(lst, lst_num))

##########df_new_used[columns_age] = df_used[columns_age].replace(age_dict) # REACTIVATE
analyse_column_list(columns_age,df_new_used,show=False)

In [ ]:
%%time
df_temp_2 = pd.DataFrame()
df_temp_2['ACTUALAGE_SUBJ_original'] = df_used['ACTUALAGE_SUBJ'].map(age_dict.get)
df_temp_2[date_columns] = df_new_used[date_columns].copy()
#print(df_temp_2.head())
counter = df_temp_2.count()




if len(list(counter)) == 1:
    print("a")
    date_column = date_columns[0]
else:
    len_check = (list(counter)[1]) > (list(counter)[0])
    if len_check:
        print("b")
        date_column = date_columns[0]
    else:
        print("c")
        date_column = date_columns[1]
print(date_column)
#print(df_temp_2.head(20))

# do the transformations
#df_temp_2["Year"] = df_temp_2[date_column].dt.year

df_temp_2["Year"] = pd.DatetimeIndex(df_temp_2[date_column]).year


df_temp_2.loc[df_temp_2["ACTUALAGE_SUBJ_original"] > 1000.,"col2" ] = df_temp_2['Year']- df_temp_2["ACTUALAGE_SUBJ_original"]
df_temp_2["col3"] = df_temp_2["col2"].fillna(df_temp_2["ACTUALAGE_SUBJ_original"])
df_temp_2['col4'] = df_temp_2['col3'].where((0 <= df_temp_2['col3']) & (df_temp_2['col3'] <= 250)) # limit age to 0-250

df_new_used["ACTUALAGE_SUBJ"] = df_temp_2['col4']

In [ ]:
df_temp_2.head()

### Check and transform bathroom columns 

In [ ]:
# Clean up 'BATHS_SUBJ', 'BLGRDBATHRMS_SUBJ'
def clean_bathrooms(columns_list, source_df,which_col):
    # gets list of column names with bathrooms and source df
    # returns 1 full bathroom and 1 half bathroom column for each input column
    # specify "full" or "half" in which_col to select whihc column to transform
    cols_to_use = column_name_adjuster(columns_list, source_df)
    lst = np.unique(source_df[cols_to_use].values)

    # clean up and convert all combinations of baths into float
    import re
    lst_num = []
    for no in lst:
            temp_list = re.findall(r'[\d\.]+', no)

            if len(temp_list)==0:
                x =(np.NaN)
            else: 
                x =(temp_list[0])
            lst_num.append(float(x))
    #print(lst_num)

    # Deal with half baths
    half_bath_list =[]
    for string in lst_num:
        x = round(string % 1,2)
        if pd.isnull(x):
            half_bath_list.append(x)
        elif x==0:
            half_bath_list.append(0)
        elif 0 < x < 0.2:
            half_bath_list.append(1)
        elif 0.2 <=  x < 0.3:
            half_bath_list.append(2)
        elif 0.3 <= x < 0.4:
            half_bath_list.append(3)
        elif 0.4 <= x < 0.5:
            half_bath_list.append(4)
        else:
            half_bath_list.append(1) # anything above 0.5 is assumed to be 1 half bath

        #print(string , round((string % 1.0),2), half_bath_list[-1])  
    #print(half_bath_list)

    #deal with full baths
    full_bath_list =[]
    for string in lst_num:

        if pd.isnull(string):
            full_bath_list.append(string)
        else:
            full_bath_list.append(int(string))
        #print(string ,  full_bath_list[-1])  
    #print(full_bath_list)

    full_bath_dict = dict(zip(lst, full_bath_list))
    half_bath_dict = dict(zip(lst, half_bath_list))
    #print(full_bath_dict)
    # create new columns
    df_temp = source_df[cols_to_use]
    
    if which_col == "full":
        for j in cols_to_use:
            df_temp["FULL_"+j] = source_df[j].map(full_bath_dict.get)
            #df_temp["HALF_"+j] = source_df[j].map(half_bath_dict.get)
        #print(df_temp.head(25))
        #print(df_temp.columns.str.contains(r'full', case=False))
        #print(df_temp[df_temp.columns[df_temp.columns.str.contains(r'full', case=False)]].head())
        return (df_temp[df_temp.columns[df_temp.columns.str.contains(r'full', case=False)]])
    
    elif which_col == "half":
        for j in cols_to_use:
            #df_temp["FULL_"+j] = source_df[j].map(full_bath_dict.get)
            df_temp["HALF_"+j] = source_df[j].map(half_bath_dict.get)
        #print(df_temp.head(25))
        #print(df_temp.columns.str.contains(r'half', case=False))
        #print(df_temp[df_temp.columns[df_temp.columns.str.contains(r'half', case=False)]].head())
        return (df_temp[df_temp.columns[df_temp.columns.str.contains(r'half', case=False)]])
    else:
        print("Full or Half not specified!! Exited")
        return None
    
    

In [ ]:
# add new bathroom columns to used dataframe 

zzz_full = clean_bathrooms(['BATHS', 'BLGRDBATHRMS'],df_used,"full")
df_new_used[zzz_full.columns] = zzz_full

In [ ]:
zzz_half = clean_bathrooms(['BATHS', 'BLGRDBATHRMS'],df_used,"half")
df_new_used[zzz_half.columns] = zzz_half

### Check and transform site (area sqft/ac)

In [ ]:
df_temp=pd.DataFrame()
df_temp["SITE_SUBJ"] = df_used[site_columns[0]]
df_temp["no_comma"] = df_temp["SITE_SUBJ"].str.replace(',', '')
df_temp['extracted_Number'] = df_temp['no_comma'].str.replace('([A-Za-z]+)', '',regex=True)
df_temp['extracted_Letter'] = df_temp['no_comma'].str.extract('([A-Za-z]+)')
df_temp['extracted_Letter'] = df_temp['extracted_Letter'].str.lower()

#clean up the extracted_number field

lst = df_temp['extracted_Number'].unique()
#print(len(lst))
import re
lst_num = []
for no in lst:
    
    try:
        test = float(no)
        lst_num.append(test)
    except Exception as e: 
        #print(no)
        #x = no.lstrip()
        x = no.lstrip('0')
        #print(x)
        temp_list = re.findall(r'[+-]?([1-9]\d*(\.\d*[1-9])?|0\.\d*[1-9]+)|\d+(\.\d*[1-9])?', x)#look for numbers
        if len(temp_list)==0:
            lst_num.append(np.nan)
        else:
            #print(temp_list)
            try:
                lst_num.append(float(temp_list[0][0]))#take first number
            except Exception as e: 
                lst_num.append(float(temp_list[1][0]))#take first number

site_dict = dict(zip(lst, lst_num))

df_temp["extracted_Number_clean"] = df_temp["extracted_Number"].map(site_dict.get) #QUICKEST!!!!
#analyse_column_list(["extracted_Number_clean"],df_temp)

# Clean up number and Convert ac to sqft 
df_temp['extracted_Letter_with_nan'] = df_temp['extracted_Letter']
area_unit_dict = {"sf": 1., "s": 1., "sq": 1., "sqft": 1., "ac": 43560.}
df_temp['conversion_rate'] = df_temp['extracted_Letter'].map(area_unit_dict).fillna(1.0) # assuming any populated areas without unit are sqft
df_temp['SITEAREASQFT'] = df_temp['extracted_Number_clean'].mul(df_temp['conversion_rate'])

# Add "SITEAREASQFT" to df_used
df_new_used['SITEAREASQFT_SUBJ'] = df_temp['SITEAREASQFT']

## Apply other last minute transformations

In [ ]:
# Fill NA in Belowground

bldgrd = df_new_used.columns[df_new_used.columns.str.contains(r'blgrd', case=False)]
df_new_used[bldgrd] = df_new_used[bldgrd].fillna(0.)

In [ ]:
# Remove _SUBJ suffix

df_new_used.columns = df_new_used.columns.str.rstrip('_SUBJ')

In [ ]:
df_new_used.columns

## Save the processsed dataframes to pickle

In [ ]:
if which_df == "APPRAISAL":
    df_new_used.to_pickle("./OUT_dfs/df_appr_full_processed.pkl")
    print("!!!!!!!!!!!!!APPRAISAL SAVED!!!!!!!!!!!!!!!")
elif which_df == "COMPARABLES":
    df_new_used.to_pickle("./OUT_dfs/df_comp_full_processed.pkl") # column names have been changed to be same as in APPR df
    print("!!!!!!!!!!!!!COMP SAVED!!!!!!!!!!!!!!!")
else:
    raise ValueError("############Incorrect dataframe selected#############")
